In [1]:
# when changing riders, do a factory reset and give the folder key inside file list
!pip install -U -q PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# choose a local (colab) directory to store the data.
local_download_path = os.path.expanduser('~/data')
try:
  os.makedirs(local_download_path)
except: pass

# 2. Auto-iterate using the query syntax
#    https://developers.google.com/drive/v2/web/search-parameters
file_list = drive.ListFile(
    {'q': "'1s9nJuoaHfKdspaEuM4GVWr7IvIIk2AcL' in parents and trashed = False and mimeType != 'application/vnd.google-apps.folder'"}).GetList() #import swd files

for f in file_list:
  # 3. Create & download by id.
  print('title: %s, id: %s' % (f['title'], f['id']))
  fname = os.path.join(local_download_path, f['title'])
  print('downloading to {}'.format(fname))
  f_ = drive.CreateFile({'id': f['id']})
  f_.GetContentFile(fname)

title: rider1_ride4_timed10.csv, id: 1pnZ6Jj8rcunHb0ZKnoBR00Fg_kny13G5
downloading to /root/data/rider1_ride4_timed10.csv
title: rider1_ride3_timed10.csv, id: 1izLX4U0D3ZtuajBZ0YGEKS9J8xOgEm_p
downloading to /root/data/rider1_ride3_timed10.csv
title: rider1_ride2_timed10.csv, id: 1VRG_f8JD6EfJ_-huF7jCLr3zwZuPTEl0
downloading to /root/data/rider1_ride2_timed10.csv
title: rider1_ride1_timed10.csv, id: 15wMuHMcG9OC8goI6MSXvpdaaC1tI7Hwg
downloading to /root/data/rider1_ride1_timed10.csv


In [2]:
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import dates as mdate
import numpy as np
import math
from datetime import datetime
from google.colab import files

import statsmodels.api as sm
from scipy import stats

#import fbprophet
import seaborn as seabornInstance 
from sklearn.model_selection import train_test_split 
from sklearn import linear_model 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn import preprocessing
from sklearn import svm
from sklearn.kernel_approximation import Nystroem

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:


import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import glob 
import copy
#tf.compat.v1.enable_eager_execution() 

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

#read csv all data, rename the old files with _old
#all_files = glob.glob("//root//data//*raw.csv")
#test_files = glob.glob("//root//data//*old.csv")

#read csv rider wise, rename the test file as test_ and keep the rider files as is
all_files = glob.glob("//root//data//rider*.csv")
test_files = glob.glob("//root//data//test*.csv") # rename a file for testing as test_ in root/data or root/data2

li=[]
for filename in all_files:
    print('importing...',filename)
    df1 = pd.read_csv(filename, index_col=None, header=0)
    print('before drop',len(df1))
    df1.replace(r"^\s*$",np.nan,inplace=True, regex=True)
    df1.replace(r"^\s*true$",1,inplace=True, regex=True)
    df1.replace(r"^\s*false$",0,inplace=True, regex=True)
    df1.dropna(axis=0,how='any',inplace=True)
    print('after drop',len(df1))
    li.append(df1)

df = pd.concat(li, axis=0, ignore_index=True)
total_trn = len(df)
print('total_train',len(df))

li1=[]
for filename in test_files:
  print('importing...',filename)
  df1=pd.read_csv(filename, index_col=None, header=0)
  #df1[['cadence','altitude_smooth','grade_smooth','velocity_smooth','watts']].str.strip()
  print('before drop1',len(df1))
  df1.replace(r"^\s*$",np.nan,inplace=True, regex=True)
  df1.replace(r"^\s*true$",1,inplace=True, regex=True)
  df1.replace(r"^\s*false$",0,inplace=True, regex=True)
  #df1.replace("",np.NaN,inplace=True)
  #print(df1['watts'].head())
  df1.dropna(axis=0,how='any',inplace=True)
  #df1.to_csv('new3182519785_dropped.csv')
  #files.download('new3182519785_dropped.csv')
  print('after drop1',len(df1))
  li1.append(df1)

test_df = pd.concat(li1, axis=0, ignore_index=True)
test_df_copy = copy.deepcopy(test_df)
#test_df=test_df.dropna()
total_tst = len(test_df)
print('total_test',total_tst)
#print(test_df['watts'].empty())
#test_df_filtered = test_df[test_df['watts'].str.strip()!=np.NaN]
print(test_df.tail())
#df1 = pd.read_csv('_5.csv')
#df=df.append(df1, ignore_index=True)

print (test_df.isnull().any())
print(test_df.columns)

importing... //root//data/rider1_ride4_timed10.csv
before drop 663
after drop 663
importing... //root//data/rider1_ride2_timed10.csv
before drop 1330
after drop 1330
importing... //root//data/rider1_ride1_timed10.csv
before drop 1369
after drop 1369
total_train 3362
importing... //root//data/test_rider1_ride3_timed10.csv
before drop1 231
after drop1 231
total_test 231
     Unnamed: 0  Unnamed: 0.1  ...  accel_sensor(m2/s)  accel_gps(m2/s)
226         226        1138.5  ...            0.004493         0.002644
227         227        1143.5  ...            0.003991         0.002848
228         228        1148.5  ...            0.005808         0.003444
229         229        1153.5  ...            0.008122         0.003439
230         230        1158.5  ...            0.007741         0.003458

[5 rows x 35 columns]
Unnamed: 0                   False
Unnamed: 0.1                 False
time                         False
timer_time                   False
distance                     False

In [4]:


#minimal and extended feature headers. if you normalize and then use the data, remove weight from features, because it is a column of constants and will break the model.

# for all files, make sure you have the same no. of cols in old and new and in same order

header_sensor_minimal_new = ['distance','speed_from_sensor(m/s)','accel_sensor(m2/s)','grade_smooth','weight_bike_rider_80','watts']#'weight_bike_rider_80',
header_gps_minimal_new = ['distance_calc(m)','speed_from_gps(m/s)','accel_gps(m2/s)','grade_smooth','watts']#'weight_bike_rider_80',

header_sensor_extended_new = ['distance','speed_from_sensor(m/s)','accel_sensor(m2/s)','grade_smooth','wind_resistance_sensor(N)',
                              'temp','cadence','altitude','watts']#'weight_bike_rider_80','heartrate',
header_gps_extended_new = ['distance_calc(m)','speed_from_gps(m/s)','accel_gps(m2/s)','grade_smooth','wind_resistance_gps(N)',
                           'temp','cadence','altitude','watts']#'weight_bike_rider_80','heartrate',
header_sensor_minimal_old = ['distance(m)','speed(m/s)','accel_sensor(m2/s)','grade(%)','watts' ] #,'weight_bike_rider_80'
header_gps_minimal_old = ['distance_calc(m)','speed_from_gps(m/s)','accel_gps(m2/s)','grade(%)', 'watts' ]#,'weight_bike_rider_80'

header_sensor_extended_old = ['distance(m)','speed(m/s)','accel_sensor(m2/s)','grade(%)','wind_resistance_sensor(N)',
                              'temperature(C)','cadence(rpm)','altitude(m)','watts']#'weight_bike_rider_80',
header_gps_extended_old = ['distance_calc(m)','speed_from_gps(m/s)','accel_gps(m2/s)','grade(%)','wind_resistance_gps(N)',
                           'temperature(C)','cadence(rpm)','altitude(m)','watts']#'weight_bike_rider_80',

features_considered = header_sensor_minimal_new # change header here to switch between min and extended, if riderwise then put the same header, if all files put new, then old

features = df[features_considered]

print(features.head())

test_features_considered = header_sensor_minimal_new # change header here to switch between min and extended, if riderwise then put the same header, if all files put new, then old


test_features=test_df[test_features_considered]
test_features.columns=features_considered 

print(test_features.tail())

frames = [features,test_features] 
features = pd.concat(frames)  
features[features_considered] = features[features_considered].astype(float) 

# below code to be uncommented on for all data

#index1 = total_trn - 7943  # switch on when all files - last fle of rider 3
#index2 = total_trn + 7195  # switch on when all files - first file of rider 4
#train1 = features.iloc[:index1,:] # switch on when all files
#test = features.iloc[index1:index2, :] # switch on when all files
#train2 = features.iloc[index2:,:]# switch on when all files
#frames2 = [train1,train2,test] # switch on when all files - used when all data
#features = pd.concat(frames2)  # switch on when all files - used when all data
#total_tst = len(test)
#print(total_tst)


print(features.head())
print(features.tail())
print('total dataset',len(features))


#print(features[~features.applymap(np.isreal).all(1)])
#plotting individual graphs for features
#features.plot(subplots=True)
#plt.legend(loc=1)
#plt.savefig('features_plot.png')
#files.download("features_plot.png") 

#plt.show()

     distance  speed_from_sensor(m/s)  ...  weight_bike_rider_80       watts
0   57.900000                6.660000  ...                  80.0  219.600000
1   90.530000                6.390000  ...                  80.0  171.750000
2  123.743333                6.616667  ...                  80.0  155.633333
3  152.257500                5.912500  ...                  80.0  145.700000
4  180.612000                6.066000  ...                  80.0  167.000000

[5 rows x 6 columns]
         distance  speed_from_sensor(m/s)  ...  weight_bike_rider_80       watts
226  11617.122952               10.395551  ...                  80.0  265.926872
227  11670.002500               10.400614  ...                  80.0  266.105263
228  11722.939694               10.409956  ...                  80.0  266.310917
229  11775.959870               10.404993  ...                  80.0  266.241304
230  11828.983377               10.408348  ...                  80.0  266.387013

[5 rows x 6 columns]
     dis

In [5]:
#runs at all times to get the results
#please re-run the block above this block before you run this block, otherwise it will fail with an nd-array has no values error

#train_split = len(train1)+len(train2) # uncomment only when all files are used.
train_split=total_trn # uncomment when rider wise data is used

#data normalization
'''
features=features.values.astype('float')  #turn on when u want to normalize ( do not use for GLM)
data_mean=features[:train_split].mean(axis=0)  #turn on when u want to normalize ( do not use for GLM)
data_std=features[:train_split].std(axis=0)  #turn on when u want to normalize ( do not use for GLM)

data_min=features[:train_split].min(axis=0)  #turn on when u want to normalize ( do not use for GLM)
print('data_mean', data_mean, 'data_std',data_std)  #turn on when u want to normalize ( do not use for GLM)
features=(features-data_mean)/data_std  #turn on when u want to normalize ( do not use for GLM)

features_train = features[:train_split] #used when data normalized
features_test = features[train_split:] #used when data normalized, if minimal use [:,:4], if extended use [:,:8] 
X_train = features_train[:,:4].astype('float') #used when data normalized
#print(X_train[0])
X_test = features_test[:,:4].astype('float') #used when data normalized, if minimal use [:,:4], if extended use [:,:8] 
y_train = features_train[:,-1].astype('float') #used when data normalized
y_test = features_test[:,-1].astype('float') #used when data normalized
y_test=(y_test*data_std[-1]+data_mean[-1])
'''

#uncomment when non-normalized data is to be used
features_train = features[:train_split] #used when data is non-normalized
features_test = features[train_split:] #used when data is non-normalized
X_train = features_train.iloc[:,:4].values.astype('float') #used when data is non-normalized, if minimal use [:,:4], if extended use [:,:8] 
#print(X_train[0])
X_test = features_test.iloc[:,:4].values.astype('float') #used when data is non-normalized, if minimal use [:,:4], if extended use [:,:8] 
y_train = features_train.iloc[:,-1].values.astype('float') #used when data is non-normalized
y_test = features_test.iloc[:,-1].values.astype('float') #used when data is non-normalized
y_watts_calc = features_test.iloc[:,-2].values.astype('float')



#use with riders 1 to 3 and uncomment when you have uncommented the data normalization above. Keep commented when data is non-normalized
#y_watts_calc=y_watts_calc*data_std[-2]+data_mean[-2]
#print(y_train[0])



#print(len(features_train),len(features_test))
y_pred_svr=[]
y_pred_glm=[]
y_pred_linear=[]
y_pred_lasso=[]
print('mean y_test',np.mean(y_test))

model=['linear','lasso','svr', 'GLM'] #remove 'GLM' from this list if data is normalized, else keep 'GLM'
for type in model:
  if type == 'svr':
    regressor=svm.SVR(kernel='rbf', C=1, gamma='auto', degree=5, epsilon=.1,coef0=1,  tol=1)
    '''
    regressor = linear_model.SGDRegressor()
    feature_map_nystroem = Nystroem(gamma = .2, random_state = 1, n_components = 2000)
    X_train_transformed = feature_map_nystroem.fit_transform(X_train)
    X_test_transformed = feature_map_nystroem.fit_transform(X_test)
    regressor.fit(X_train_transformed, y_train)
    y_pred = regressor.predict(X_test_transformed)
    '''
    regressor.fit(X_train,y_train)
    y_pred = regressor.predict(X_test)

    print(y_test)
    #y_pred=y_pred*data_std[-1]+data_mean[-1] #uncomment when normalized

    print('Root Mean Squared Error (SVR Regression):', metrics.mean_squared_error(y_test,y_pred,squared=False))
    print('Mean Abs Error (SVR Regression):', metrics.mean_absolute_error(y_test,y_pred))

    print('R2 Score (SVR):',metrics.r2_score(y_test,y_pred))

    print('mean y_pred (svr)', np.mean((y_pred)))
    y_pred_svr=y_pred
    #print(len(y_pred))
    continue

  if type=='linear':
    regressor = LinearRegression()  
    regressor.fit(X_train, y_train)
    y_pred = regressor.predict(X_test)
    #y_pred=y_pred*data_std[-1]+data_mean[-1] #uncomment when data is normalized
    print('Root Mean Squared Error (Linear Regression):', metrics.mean_squared_error(y_test,y_pred,squared=False))
    print('Mean Abs Error (linear Regression):', metrics.mean_absolute_error(y_test,y_pred))
    print('R2 Score (Linear):',metrics.r2_score(y_test,y_pred))
    print('mean y_pred (Linear)', np.mean((y_pred)))
    y_pred_linear=y_pred
    print(y_test)
    #LinR=pd.DataFrame({'Actual Test': y_test, 'Predicted Linear': y_pred })
    #LinR.to_csv('LinR.csv')
    #np.savetxt('RMSE_Linear.txt',  [np.sqrt(metrics.mean_squared_error(y_test, y_pred))])
    continue
  if type=='lasso':
    regressor=linear_model.Lasso(alpha=2/3, tol=0.1)
    regressor.fit(X_train, y_train)
    print(regressor.coef_)
    y_pred = regressor.predict(X_test)
    #y_pred=y_pred*data_std[-1]+data_mean[-1]  #uncomment when data is normalized
    print('Root Mean Squared Error (Lasso Regression):', metrics.mean_squared_error(y_test,y_pred,squared=False))
    print('Mean Abs Error (Lasso Regression):', metrics.mean_absolute_error(y_test,y_pred))
    print('R2 Score (Lasso):',metrics.r2_score(y_test,y_pred))
    print('mean y_pred (lasso)', np.mean((y_pred)))
    y_pred_lasso = y_pred
    print(y_test)
    #LS=pd.DataFrame({'Actual Test': y_test, 'Predicted Linear': y_pred })
    #LS.to_csv('LS.csv')
    #np.savetxt('RMSE_Lasso.txt',  [np.sqrt(metrics.mean_squared_error(y_test, y_pred))])
    continue

  if type == 'GLM': # GLM cannot be run with normalized power, because then dependent variable becomes negative
    exog, endog = sm.add_constant(X_train), y_train
    mod = sm.GLM(endog,exog,family=sm.families.Gaussian(link=sm.families.links.log()))
    res = mod.fit(maxiter=100)
    with np.errstate(divide='ignore'):
      yhat = res.predict(sm.add_constant(X_test))
    sqared_diff_sensor = 0
    for i in range(len(y_test)):
      sqared_diff_sensor = sqared_diff_sensor + (y_test[i] - yhat[i])**2

    RMSE_sensor = math.sqrt((sqared_diff_sensor)/len(y_test))
   
    #print(RMSE_sensor)
    print('Root Mean Squared Error (GLM):', metrics.mean_squared_error(y_test,yhat,squared=False))
    print('Mean Abs Error (GLM):', metrics.mean_absolute_error(y_test,yhat))
    print('R2 Score (GLM):',metrics.r2_score(y_test,yhat))
    print('mean y_pred (glm)', np.mean((yhat)))
    print(y_test)

    y_pred_glm = yhat






mean y_test 262.24827518875503
Root Mean Squared Error (Linear Regression): 21.77570173412175
Mean Abs Error (linear Regression): 20.935270240031763
R2 Score (Linear): -19.942637976338982
mean y_pred (Linear) 241.51073755788647
[270.5        267.5        269.23333333 269.475      268.88
 272.95       273.34285714 274.2        276.78888889 278.27
 277.42727273 277.275      267.19230769 254.00714286 243.08666667
 240.5875     242.48823529 243.58333333 244.45263158 245.985
 248.0047619  249.84090909 252.04782609 253.22916667 254.772
 255.87307692 256.57777778 257.07142857 257.28275862 257.46333333
 257.48709677 257.996875   258.33636364 258.82941176 259.19714286
 259.28055556 259.46486486 259.55526316 259.73846154 260.3525
 260.85853659 261.48809524 261.9255814  261.94545455 262.11111111
 262.40652174 262.16170213 261.89583333 261.36734694 260.642
 259.65686275 259.30384615 259.59622642 259.63888889 259.91272727
 259.85       259.68596491 259.41724138 259.51355932 259.51833333
 259.144262

In [ ]:
#to download the predictions - no need to run if you don't want to download

#d={'y_test':y_test,'watts_calc':y_watts_calc,'y_pred_linear':y_pred_linear,'y_pred_lasso':y_pred_lasso,'y_pred_svr':y_pred_svr,'y_pred_glm':y_pred_glm} #to be used with riders1 to 3, because they have watts_calc(strava feature in the data)
#d={'y_test':y_test,'y_pred_linear':y_pred_linear,'y_pred_lasso':y_pred_lasso,'y_pred_svr':y_pred_svr,'y_pred_glm':y_pred_glm} # to be used with rider 4
d={'y_test':y_test,'y_pred_linear':y_pred_linear,'y_pred_lasso':y_pred_lasso,'y_pred_svr':y_pred_svr} # uncomment when GLM is removed from the list above and data is normalized
pred_df=pd.DataFrame(d)
pred_df.to_csv('predict_r1_sensor_ext.csv')
files.download('predict_r1_sensor_ext.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>